<a href="https://colab.research.google.com/github/zanasaed/NoteBook/blob/main/ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Links of library and tutorial 

    [FreeCourseLab.com] Udemy - Elasticsearch 6 and Elastic Stack - In Depth and Hands On!

[sundog-education](https://sundog-education.com/elasticsearch/)


doc: [offical](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)


# How install 

look main website for [instruction](https://www.elastic.co/guide/en/elasticsearch/reference/current/deb.html) 

list of error that maybe see in instalation :

`gpg: no valid OpenPGP data found.`
[solution](https://stackoverflow.com/questions/21338721/gpg-no-valid-openpgp-data-found) 
be sure you use VPN 

[Elasticsearch: Job for elasticsearch.service failed
](https://stackoverflow.com/questions/58656747/elasticsearch-job-for-elasticsearch-service-failed)


to see elastic status `sudo systemctl status elasticsearch`


to test elastic is runing
`curl 127.0.0.1:9200`


# definition 


## Key Concepts

**Node**

It refers to a single running instance of Elasticsearch. Single physical and virtual server accommodates multiple nodes depending upon the capabilities of their physical resources like RAM, storage and processing power.

---

**Cluster**

It is a collection of one or more nodes. Cluster provides collective indexing and search capabilities across all the nodes for entire data.

---
 
**Index** (indeces) Like DataBase 

It is a collection of different type of documents and their properties. Index also uses the concept of shards to improve the performance. For example, a set of document contains data of a social networking application.

An index powers search into all documents withn a collection of types. They contain inverted indices that let you search across everything within them at once.

---

**Document** Like Row 

It is a collection of fields in a specific manner defined in JSON format. Every document belongs to a type and resides inside an index. Every document is associated with a unique identifier called the UID.

Documents are the things you're searching for.They can be more text - any structured JSON data works. Every document has a unique ID,and a type.

---

**types** Like Table 
A type define the schema and mapping shared by documents that represent the same sort of things. ( A log entry, an encyclopedia article, etc.)


**Note** : Elasticsearch is moving toward eliminating the concept of type. in Elasticsearch 6, only one type is allowed per index.

---
**Shard**

Indexes are horizontally subdivided into shards. This means each shard contains all the properties of document but contains less number of JSON objects than index. The horizontal separation makes shard an independent node, which can be store in any node. Primary shard is the original horizontal part of an index and then these primary shards are replicated into replica shards.

---


**Replicas**

Elasticsearch allows a user to create replicas of their indexes and shards. Replication not only helps in increasing the availability of data in case of failure, but also improves the performance of searching by carrying out a parallel search operation in these replicas.




# Mapping and Indexing Data


## Definition 
a maping is a schema definition.

elasticsearach has resonbale defaults, but somtimes you need to customize them.

we can use maping for more things like:

**field types** :

text, keyboard, byte, short, integer, long, float, double, boolean, data


```
"properties" : {
    "user_id" : {
        "type" : "long"
    }
}
```


**field index** : 

do you want this field to be queryable? true/false 


```
"properties" : {
    "genre" : {
        "index" : "false"
    }
}
```


**field analyzer** : 

define your tokenizer and token filter. standard / whitespace / simple / english etc. 


```
"properties" : {
    "dascription" : {
        "analyzer" : "english"
    }
}
```
**more about analyzers**

There are three things analyzer can do.

* **character filters** : 

    analyzer can 
    remove HTML encoding, convert "&" to "and" word

    The idea here is that if you apply the same analyzer to both your search 
    query and the information that's index, you can get rid of all these 
    discrepancies


* **tokenizer** : 

    spilt strings on whitespace / punctuation / non-letters 

    specifiy how a sting gets broken up into search terms that get indexed. 

* **token filter** : 
 
    lowercasing, stemming, synonyms, stopwords

    if you want your search be in case sensetive you might want token filter to 
    lowecase everything

    you can also handel stemming. for example if you want the box and boxes and 
    boxing and boxed to all match for a search for the term box, you can do the 
    stemming to normalize all those diffrent variant of the given word to the 
    same root stem

    synonyms : if you want to swarch for the word big to also tuen up document 
    that contain the word large, your token filter could normalize all those to
    a given synonym, or to make sure that those also work as well.

    stop word : if you don't wnat to waste space storing thingd like (the, and,
    a , that ,etc..), you might want to apply a stop word filter as well to your
    searches, so you don't end up indexing all that stuff too.


**choices for analyzers**

* **standard** : 

    spilt on word boundaries, remove punctuation, lowercases. good choice if
    language is unknown

* **simple** : 

    split on anything that isn't a letter, and lowercases

* **whitesoace** : 

    split on whitespace but doesn't lowercase

* **language**(i.e. english) : 

    accounts for language-specific stopwords and stemming


**NOTE**: Sometimes you can even mix and match different languages in the same index. there are some tricks you can do to actualy run diffrent analyzer on the same text, and store those as diffrent fields. so i could, for example have an English vertion and a french version of the information in my index. that i can search separtely.


## example

we are using curl to send an HTTP request with a content type header that indicates JSON data.

 a PUT action to our localhost on port 9200, affecting the movies index, and we're defining  an explicit mapping for the movie type that change the property of the year to be interpreted  as a date type.


```
curl -H "Content-Type: application/json" -XPUT 127.0.0.1:9200/movies -d'
{
    "mappings" : {
        "movie" : {
             "properties" : {
                "year" : { 
                    "type": "date"
                    }
            }
        }
    }
}'
```
-H --> content type header 

-XPUT --> PUT portocol 

127.0.0.1 elasticsearch addresses in our system

:9200 --> elastic port 

/movies --> movies indexs (database)


-d' --> tells curl that the following lines are gonna be sent as the body of the HTTP request. and body should be JSON data.

**Note** : to press tab space in curl body  press ctrl+v then enter tab button 

Now if you want to retrieve that, and make sure it actually took, we can use a get action to get that mapping back, and take look at it.
```
curl -H "Content-Type: application/json" -XGET 127.0.0.1:9200/movies/_mapping/movie?pretty
```
_mapping/movie --> to retrive the mapping associated with that index,specifically for the movie type 

?pretty --> return response in a pretty formatted response.

## Hacking CURL (Don't Skip This!)

**Instructor**: One of the things that changed
in Elasticsearch six is that every request that goes to your Elasticsearch server needs to have a content header that specifies a json content-type.
And typing that in every time we issue a curl command to communicate with our server can get very tedious.

So let me show you a little hack that let's you avoid doing that.

we're going to create our own curl script that lives in bin directory, and this curl script is going to override the one built into the system.

So it's gonna be a little bash script that just calls the real curl script under user bin curl, but automatically passes in that `-H "Content-Type application/json"` parameter for us, so we don't have to keep typing it over and over and over again. And that dollar sign at sign at the end of the line there within our script says go ahead and append all the other parameters that we had on the command line for that curl command.

without this hack in place, you will need to add `-H "Content-Type application/json"` to every single curl command in this course.

In the real world, you're not gonna be using curl for the most part anyway.

You're gonna be communicating with your Elasticsearch server using some sort of coding library or some sort of larger framework. Generally speaking, you don't wanna be sneaking around behind the back of your system and making commands silently do things that they weren't designed to do originally. That can become very confusing if you were to try to issue a curl command that required a different content-type header.

You'd be left wondering why is it not doing what I'm telling it to do if you didn't remember that you had this thing in place.

<br>
<br>
Let's work on this together and get it in place. 


```
From your home directory:
mkdir bin
cd bin 
vi curl(Hit I fir insert mode)

#!/bin/bash
/usr/bin/curl -H "Content-Type: application/json" "$@"

Esc - :wq - enter 

chmod a+x curl #gives executable permissions to everybody


```

the neat thing about Ubuntu is that any scripts that are in your home directory's bin directory will take precedence over other scripts on your system.

So this how we're going to trick Ubuntu into using our version of curl instead of the default one.





Basically it takes all the parameters to the curl command and sticks it on the end of the real curl command in our system, but automatically sticks in that dash h content-type application json header for us, so we never have to type it again.

Now to make it an executable script,

there's one more thing we need to do.

chmod, for change mod, that gives executable permissions to everybody,

to make sure that it's working properly, in home directory if we type in which curl, it will tell us the path that the curl script will be executed from.



##  Import a Single Document  via JSON  REST


```
curl -XPUT 1270.0.0.2:9200/movies/movie/109487 -d'
{"genre":["IMAX","Sci-Fi"],
"title" : "Insterstellar",
"year" : 2014
}'
```


##  Import a Many Documents  via JSON  REST


```
curl -XPUT 127.0.0.1:9200/_bulk -d'
{"create": {"_index" : "movies" , "_type" : "Movie" , "_id" : "135569"}}
{"_id": "135569", "title" : "Star Trek Beyond", "year" : 2016}
{"create": {"_index" : "movies" , "_type" : "Movie" , "_id" : "109487"}}
{"_id": "109487", "title" : "Insterstellar", "year" : 2014}
{"create": {"_index" : "movies" , "_type" : "Movie" , "_id" : "58559"}}
{"_id": "58559", "title" : "Dark knight, the", "year" : 2008}

```

we can also use json file that has this row and insert it in indeses 

```
curl -XPUT 127.0.0.1:9200/_bulk?pretty --data-binary @myfile.json 
```



##  Updating Data in Elasticsearch

Every document has a _version field 

Elasticsearch documents are immutable.

When you update an existing document :

a new document is created whit a incremented _version the old document is marked for deletion


**Curl format**:

```
curl -XPOST 127.0.0.1:9200/movies/movie/109487/_update -d ' 
{
    "doc" : {
        "title": "Interseteller"
    }
}'
```


## Deleting Data in Elasticsearch
 
 for deleting doc that you know the id  just use :

 ` curl -XDELETE 127.0.0.1:9200/movies/movie/58559`

## Dealing with Concurrency

to pervent update data from multi user at the same time we can use 

Optimistic concurrency control in action 

1. set the version of doc in the update command 
2. ues retry_on_confilict in update command 

``` 
curl -XPOST 127.0.0.1:9200/movies/movie/id/_update?retry_on_conflict=5 -d '
{
    "doc" : {
        "title" : "blublu blu"
    }
}'

```
retry_on_conflict=5 : Hoe many time retry before giveing up 

```

curl -XPOST 127.0.0.1:9200/movies/movie/id/_update?version=3 -d '
{
    "doc" : {
        "title" : "blublu blu"
    }
}'


```

version=3 : doc version of the last see that we know 
for get doc version look at the doc by
 
`curl -XGET 127.0.0.1:9200/movies/movie/id?pretty`


# **Search**

## **query lite** 


for search simple and fast we can usae this format query in web browser 


```
INDEX/TYPE/_search?q=title:str
movies/move/_search?q=+year:>2010+title:trek
```

example
return all seller that kind it is seller_data
```
http://localhost:9200/digikala/seller/_search?q=kind:seller_data&pretty
```
return all movies that cotains star in the title 
```
curl -X GET "localhost:9200/movies/movie/_search?q=title:star&pretty"
```

returns all movie that have word trek in title and released after the year 2010
```
curl -X GET "localhost:9200/movies/movie/_search?q=%2Byear%3A%3E2010%2Btitle%3Atrek&pretty"
```
+year:> --> %2Byear%3A%3E
%2B it is the hexadecimal code for the ascii value of plus "+"
in Kibana : 

```
GET digikala/seller/_search?
```


`curl -X GET "localhost:9200/digikala/seller/_search?q=kind:seller_data"`


The problem is when you're sending across urls you need to encode them very often and that can make things very confusing very quickly.


So you couldn't actually reliably send across that string as is as part of a rest request you would have to url rncoded it in order to securely transmit it over the Internet. And that means that any special characters need to be encoded into their hexadecimal equivalency or encoded out.

<br>

If you want **learn more** just search for **URI  search**.
this is formally caleed "URL Search". search for that on the Elasticsearch documentation. 

it's really quite powerful, but again is only appropriate for quick "curl tests".

<br>

Reasons  why you should not use this little shortcut in production for sure :

*   cryptic and tough to debug 
*   can be a security issue if exposed end users 
*   fragile- one wrong character and you're hosed.




**Counting number of documents using Elasticsearch**
[stacklink](https://stackoverflow.com/questions/25739888/counting-number-of-documents-using-elasticsearch)  
query in terminal : 

```
curl http://localhost:9200/_cat/indices?v
```



query in Kibana :



```
GET /_cat/indices?v
```



## Using JSON Search 

insted useing url search we using json search that it is better 



**get requests can have a body**

Usually, when you talk about GET requests on HTTP requests you're talking about just retrieving a web page from a web server but you can actually send a body along with a GET request as well.
Things you can do within a query.You can have two different things.
1. filters
2. queries

There are filters and then there are queries so queries are usually used for returning data in terms of relevance.

When we're doing something like searching for the search term track you would use a query because you want to get back results in orders of relevance as to how relevant the term truck was to that given document.


However, if you have a binary operation where the answer is basically yes or no you want to use a filter instead.So filters are much more efficient than queries.

Because not only are they faster but the results can be cached by elasticsearch or so that if you do another query using the same filter they can get the results even faster.


### Example of boolean query with a filter 
we have a query that contains a boolean expression where you must have trek in the title and you also must have the filter pass the condition of the year being greater than or equal to 2010.
```
curl http://localhost:9200/movies/movie/_search?pretty -d' 
{
    "query":{
        "bool":{
            "must":{"term":{"title":"trek"}},
            "filter":{"range":{"year":{"get":2010}}}
        }
    }
}'

```


### **some types of filetrs**



there are many different kinds of filters. Range is just one of them.





*   **term**: filter by exact value 

    if you need to do filter by some exact value of a term

    `{"term":{"year":2014}}`
*   **terms**: match if any exact values in a list match 

     if you want to match anything in a list of values
     
    `{"terms":{"genre":['Sci-Fi", "Sdventure"]}}`
*  **range**: Find numbers or dates in a  given range(gt,gte,lt,lte)

     if you want to just do a greater than or less than or greater than or     
     equal or less than or equal sort of operation 

    `{"range"{"year": {"gte": 2010}}}`
*  **exisits** : Find documents where a fild exists 

    you can just test if a given field exists at all within a given document.

    `{"exists" : {"fielsd": "tags"}}
*   **missing**: Find documents where a field is missing 

    If you want to find documents where they're given field does not exist 

    `{"missing": {"firld": "tags"}}
*  **bool**: Combine filters with Boolean logic(must,must_not,should)

 we looked at a must example that's basically the equivalent of. And there is also a must not which is the equivalent of the Boolean operation not in this also a should which is the boolean equivalent of or. So you can do complex boolean filters where you can have conditions that must be combined you know with and relationships or relationships or not relationships and whatever you can imagine using a bool filter.




### **some types of queries**



* match_all: returns all documents and is the default. Normally uesd with a filter.

    `{"match_all":{}}`

* match: search analyzed result, such as full text search.

    `{"mathch": {"title": "star"}}`

* multi_match: run the same query on multiple fields.

    `{multi_match": {"query": "star", "fields": ["title", "synopsis"]}}`

* bool: Works like a bool filter, but results are scored by relevance.

## 3- Full-Text vs. Phrase Search

Sometimes you don't want to search for individual search terms like star or trek or wars you want to search for phrases that are search terms and together in a certain order like Star Trek or Star Wars.

find all terms, in the right order.
```
curl -XGET 127.0.0.1:9200/movies/movie/_search?pretty -d'
{
    "query":{
        "match_phrase":{
            "title": "star wars"
        }
    }
}
```

order matters, but you're OK with some words begin in between the terms:
```
curl -XGET 127.0.0.1:9200/movies/movie/_search?pretty -d'
{
    "query":{
        "match_phrase":{
            "title": {"query":"star beyond", "slop":1}
        }
    }
}
```

some match result qoode be : 
stars trek beyond ,  star wars beyond , beyond star  


the slop represent how far you're willing to let a term move to satistfy a phrase(in either direction)

another example:"quick brown fox" would match "quick fox" with slop of 1.

**Proximity queries**

just use really high slop if you want get any documents that contain the words in your phrase, but want documents that have the words closer together scored higher.

```
curl -XGET 127.0.0.1:9200/movies/movie/_search?pretty -d'
{
    "query":{
        "match_phrase":{
            "title": {"query":"star beyond", "slop":100}
        }
    }
}


```


## 5-Pagination


want to return results to the end user one page at a time.


**Pagination syntax**
``` 
URL search: 
curl -XGET '127.0.0.1:9200/movies/_search?size=2&from=2&pretty'


json body search 

curl -XGET 127.0.0.1:9200/movies/_search?pretty -d'
{
    "from": 2,
    "size": 2,
    "query": 
        {"match": {"genre": "Sci-Fi"}
        }
}

```
returned everything in the index for that type for the movie type with a size of two items per page starting from Item number two.
the item start from zero so item number two it is the third item in the result list.



**beware** 

This is kind of a problem with search engines in general

* deep pagination can kill performance.
* every result must be retrieved, collected, and stored.
* enforce an upper bound on how may result you'll return to users.


So if you think about it you can't really tell a elasticsearch,Just give me 
back results.Number 10373 to 10383. That's not just going to have the 
overhead of retrieving 10 results because under the hood it still has to 
retrieve and collect and sort everything in order to figure out what result 
number 10380 is, and easy to figure out all 10300 results before that as 
well. In order to give you what those those results are for the range that 
you requested. So you can really start to kill performance when you start 
paginate ing deep into your results you should enforce an upper bound on how many results you'll return to your users. Otherwise some nasty person will abuse your system and bring your system to its knees and even Web sites like Google have upper bounds on how many results they return For this reason.
<!-- -->
``` 
curl -XGET '127.0.0.1:9200/movies/_search?size=2&pretty'
<!-- return first two result the from attribute start from zero  -->

json body search 

curl -XGET 127.0.0.1:9200/movies/_search?pretty -d'
{
    "from": 2,
    "size": 2,
    "query": 
        {"match": {"genre": "Sci-Fi"}
        }
}

```


## Index management 
to see list of tables go to [this link ](http://localhost:5601/app/management/data/index_management/indices)

# Use Elastic in scrapy 


there is a repository for use Elastick in Scrapy [link to this repository](https://github.com/jayzeng/scrapy-elasticsearch)

# Kibana


## List of query that used in kibana Dev tools 



```
GET /_cat/indices?v
```



**Rturn part of Type doc**
```
GET digikala/seller/_search?
```



**Return last record Doc base seller_int_id variable**

```

GET digikala/seller/_search
{
  "query": {
    "match_all": {}
  },
  "size": 1,
  "sort": [
    {
      "seller_int_id": {
        "order": "desc"
      }
    }
  ]
}
```





ELASTICSEARCH_BUFFER_LENGTH  = 20

```
2021-06-02 17:36:21 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 5700,
 'downloader/request_count': 20,
 'downloader/request_method_count/GET': 20,
 'downloader/response_bytes': 1110467,
 'downloader/response_count': 20,
 'downloader/response_status_count/200': 20,
 'elapsed_time_seconds': 22.774146,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 6, 2, 13, 6, 21, 817350),
 'item_scraped_count': 20,
 'log_count/DEBUG': 84,
 'log_count/INFO': 12,
 'log_count/WARNING': 1,
 'response_received_count': 20,
 'scheduler/dequeued': 20,
 'scheduler/dequeued/memory': 20,
 'scheduler/enqueued': 20,
 'scheduler/enqueued/memory': 20,
 'start_time': datetime.datetime(2021, 6, 2, 13, 5, 59, 43204)}
2021-06-02 17:36:21 [scrapy.core.engine] INFO: Spider closed (finished)

```
ELASTICSEARCH_BUFFER_LENGTH  = 10

```

2021-06-02 17:44:28 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 28500,
 'downloader/request_count': 100,
 'downloader/request_method_count/GET': 100,
 'downloader/response_bytes': 5533242,
 'downloader/response_count': 100,
 'downloader/response_status_count/200': 100,
 'elapsed_time_seconds': 124.430781,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 6, 2, 13, 14, 28, 1013),
 'item_scraped_count': 100,
 'log_count/DEBUG': 407,
 'log_count/INFO': 19,
 'log_count/WARNING': 1,
 'response_received_count': 100,
 'scheduler/dequeued': 100,
 'scheduler/dequeued/memory': 100,
 'scheduler/enqueued': 100,
 'scheduler/enqueued/memory': 100,
 'start_time': datetime.datetime(2021, 6, 2, 13, 12, 23, 570232)}
2021-06-02 17:44:28 [scrapy.core.engine] INFO: Spider closed (finished)

```

ELASTICSEARCH_BUFFER_LENGTH  = 50

```
2021-06-02 18:50:17 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 85500,
 'downloader/request_count': 300,
 'downloader/request_method_count/GET': 300,
 'downloader/response_bytes': 16403178,
 'downloader/response_count': 300,
 'downloader/response_status_count/200': 299,
 'downloader/response_status_count/404': 1,
 'elapsed_time_seconds': 365.304469,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 6, 2, 14, 20, 17, 760053),
 'item_scraped_count': 300,
 'log_count/DEBUG': 1219,
 'log_count/INFO': 35,
 'log_count/WARNING': 1,
 'response_received_count': 300,
 'scheduler/dequeued': 300,
 'scheduler/dequeued/memory': 300,
 'scheduler/enqueued': 300,
 'scheduler/enqueued/memory': 300,
 'start_time': datetime.datetime(2021, 6, 2, 14, 14, 12, 455584)}
2021-06-02 18:50:17 [scrapy.core.engine] INFO: Spider closed (finished)

```

ELASTICSEARCH_BUFFER_LENGTH  = 100

```
2021-06-02 20:20:35 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 285000,
 'downloader/request_count': 1000,
 'downloader/request_method_count/GET': 1000,
 'downloader/response_bytes': 53706790,
 'downloader/response_count': 1000,
 'downloader/response_status_count/200': 998,
 'downloader/response_status_count/404': 2,
 'elapsed_time_seconds': 1213.163875,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 6, 2, 15, 50, 35, 383917),
 'item_scraped_count': 1000,
 'log_count/DEBUG': 4031,
 'log_count/INFO': 81,
 'log_count/WARNING': 1,
 'response_received_count': 1000,
 'scheduler/dequeued': 1000,
 'scheduler/dequeued/memory': 1000,
 'scheduler/enqueued': 1000,
 'scheduler/enqueued/memory': 1000,
 'start_time': datetime.datetime(2021, 6, 2, 15, 30, 22, 220042)}
2021-06-02 20:20:35 [scrapy.core.engine] INFO: Spider closed (finished)

```